In [0]:
%run ./00_config

In [0]:
# ---------- Load the data from bronze ----------

df = spark.read.option("multiline", "true").json(f"{bronze_adls}/{ingestion_date}_{sport_name}_season_data.json")

In [0]:
# ---------- Clean and filter the data ----------
correct_league_id = sports_ids[sport_name]

cleaned_df = (df
    .filter((col("strSport") == "Fighting") & (col("idLeague") == correct_league_id))  
    .dropDuplicates(["idEvent"]) 
    .withColumn("dateEvent", to_date(col("dateEvent"), "yyyy-MM-dd")) 
    .withColumn("strResult", when(col("strResult").isNull(), "Unknown").otherwise(col("strResult"))) 
    .select("idEvent", "strEvent", "strPostponed", "dateEvent", "strVenue", "strCity", "strCountry", "idLeague", "strSport", "strDescriptionEN").withColumn("postponed_flag", when(col("strPostponed") == "yes", True).otherwise(False))
)

In [0]:
# ---------- Saving the cleaned data ----------

cleaned_df.write.mode("append").parquet(silver_adls)